In [1]:
# %load_ext autoreload
# %autoreload 2

In [3]:
from tqdm import tqdm
from path import Path
import json
import time
from pprint import pprint
import re

import numpy as np
import pandas as pd
%matplotlib inline
%precision 6


from collections import OrderedDict
import logging
import tempfile
import os

basedir=Path(r'/media/isisilon/Data/My_Documents/Documents/eclipse-workspace/bb2/mhwebui2/mhwebui2/scripts/')


# import makehuman-cmd

https://bitbucket.org/duststorm01/makehuman-commandline

In [4]:
# from .mh_helpers import clean, short_hash, clean_modifier

mhpath = Path(os.path.abspath("../vendor/makehuman-commandline/makehuman"))

#===============================================================================
# Import Makehuman resources, needs to be with makehuman dir as current dir
#===============================================================================

appcwd = os.path.abspath(os.curdir)
sys.path.append(mhpath)
sys.path.append(appcwd)
sys.path.append('.')

def getHuman():
    """Load a human model with modifiers."""
    with mhpath:
        # maxFaces *uint* Number of faces per vertex (pole), None for default (min 4)
        human = Human(files3d.loadMesh(
            getpath.getSysDataPath("3dobjs/base.obj"),
            maxFaces=5))
        # load modifiers onto human
        humanmodifier.mods_loaded = False
        modifiers = humanmodifier.loadModifiers(
            getpath.getSysDataPath('modifiers/modeling_modifiers.json'), human)
        return human

with mhpath:
    import makehuman
    oldpath = os.sys.path
    makehuman.set_sys_path()
    # make makehuman paths absolute by going through newest paths and making abs
    for i in range(len(os.sys.path)):
        p = os.sys.path[i]
        if p[0:2] == './':
            os.sys.path[i] = os.path.join(
                os.path.abspath('.'), p.replace('./', ''))
        else:
            break

    makehuman.init_logging()
    logging.getLogger().setLevel(logging.CRITICAL)
    #import image_pil as image_lib
    #
    import proxy as mhproxy
    import humanargparser
    import targets as mhtargets
    from human import Human
    import files3d
    import getpath
    import humanmodifier
    from core import G
    import headless
    import autoskinblender
    import export
    
    # Init console app
    with mhpath:
        G.app = headless.ConsoleApp()
    G.app.selectedHuman = human = getHuman()
    headless.OBJExporter = None
    headless.MHXExporter = None
    headless.MhxConfig = None
    humanargparser.mods_loaded = False
    
    from makehuman import LicenseInfo
    mh_licence=LicenseInfo()

def assignModifierValues(human, valuesDict):
    _tmp = human.symmetryModeEnabled
    human.symmetryModeEnabled = False
    for mName, val in valuesDict.items():
        try:
            human.getModifier(mName).setValue(val)
        except:
            pass
    human.updateMacroModifiers()
    human.applyAllTargets()
    human.symmetryModeEnabled = _tmp
    return human



Initialized logging


In [5]:
os.sys.path.append(basedir)
from convert_obj_three import convert_ascii
from export_makehuman import material_to_mtl, vertex_weights_to_skin_weights, parse_skeleton_bones, NP_MH_Encoder

# comparison of rigs

      rigfile                            amount_over  maxvi  mean_over  nb_bones  number_over  
               data/rigs/cmu_mb.mhskel          6700      8   4.376225        31         1531   
          data/rigs/game_engine.mhskel          3947      7   4.023445        53          981  
      data/rigs/default_no_toes.mhskel         20036      8   5.107316       137         3923  
              data/rigs/default.mhskel         20412      8   5.086469       163         4013   
     
    

In [6]:
# choose rig
rigfile = 'data/rigs/default.mhskel'
# rigfile = 'data/rigs/game_engine.mhskel'

In [7]:
outdir = Path(tempfile.mkdtemp(suffix='Export_base_human_to_threejs_json'))
print outdir

/tmp/tmpKpAxvHExport_base_human_to_threejs_json


In [8]:
# make base human model with helpers enabled
with mhpath: 
    basehuman = getHuman()
    
    # rig
    if rigfile:
        humanargparser.addRig(basehuman,rigfile)    
    
    mesh = basehuman.meshData
    group_mask = np.ones(len(mesh._faceGroups), dtype=bool)
    face_mask = group_mask[mesh.group]
    basehuman._staticFaceMask = face_mask
    basehuman.meshData.changeFaceMask(basehuman.staticFaceMask)
    basehuman.meshData.updateIndexBufferFaces()
    basehuman.changeVertexMask(None)

In [9]:
# export
infile = mhpath + '/data/3dobjs/base.obj'
outfile = outdir.joinpath('models','human_full_size.json')
influencesPerVertex=4

mtllib = material_to_mtl(basehuman.material, texdir=os.path.dirname(outfile))
convert_ascii(
    infile=infile,
    morphfiles='',
    colorfiles='',
    outfile=outfile,
    licence=json.dumps(mh_licence.asDict()),
#     mtllib=mtllib
)

# some extra data to add to the file
skeleton = basehuman.getSkeleton()
bones = parse_skeleton_bones(skeleton)
skeletonMetadata = {
    "name": skeleton.name,
    "version": skeleton.version,
    "description": skeleton.description,
    "license": skeleton.license.asDict(),
}
vertex_weights = basehuman.mesh.getVertexWeights(skeleton.getVertexWeights(force_remap=True))
skinIndices, skinWeights = vertex_weights_to_skin_weights(
    vertex_weights, skeleton, influencesPerVertex=influencesPerVertex)

licence = json.dumps(mh_licence.asDict())

# now add extra data to file
metadata = {
    "skeletonMetadata": skeletonMetadata,
    "joint_pos_idxs": skeleton.joint_pos_idxs
}
data = json.load(open(outfile))
data['metadata'].update(metadata)
data['skinIndices'] = skinIndices
data['skinWeights'] = skinWeights
data['bones'] = bones
data['influencesPerVertex']=influencesPerVertex

json.dump(data, open(outfile, 'w'), cls=NP_MH_Encoder, separators=(',', ':'))
outfile

19158 vertices, 18486 faces, 139 materials, 0 morph targets, 0 bones, 0 skinWeights


Path(u'/tmp/tmpKpAxvHExport_base_human_to_threejs_json/models/human_full_size.json')

In [10]:
outfile

Path(u'/tmp/tmpKpAxvHExport_base_human_to_threejs_json/models/human_full_size.json')

## Export skins

In [21]:
os.sys.path.append(basedir)
from convert_obj_three import generate_materials_string, create_materials, generate_materials
from export_makehuman import material_to_mtl, copyAndCompress

In [22]:
# import proxies
def _listDataFiles(foldername,
                   extensions,
                   onlySysData=False,
                   recursive=True):
    with mhpath:  # sadly makehuman seems hardcoded
        if onlySysData:
            paths = [getpath.getSysDataPath(foldername)]
        else:
            paths = [getpath.getDataPath(foldername),
                     getpath.getSysDataPath(foldername)]
    return list(getpath.search(paths, extensions, recursive))

In [23]:
skins = _listDataFiles('skins','mhmat')
skins

[u'data/skins/default.mhmat',
 u'data/skins/middleage_african_female/middleage_african_female.mhmat',
 u'data/skins/middleage_african_male/middleage_african_male.mhmat',
 u'data/skins/middleage_asian_female/middleage_asian_female.mhmat',
 u'data/skins/middleage_asian_male/middleage_asian_male.mhmat',
 u'data/skins/middleage_caucasian_female/middleage_caucasian_female.mhmat',
 u'data/skins/middleage_caucasian_male/middleage_caucasian_male.mhmat',
 u'data/skins/old_african_female/old_african_female.mhmat',
 u'data/skins/old_african_male/old_african_male.mhmat',
 u'data/skins/old_asian_female/old_asian_female.mhmat',
 u'data/skins/old_asian_male/old_asian_male.mhmat',
 u'data/skins/old_caucasian_female/old_caucasian_female.mhmat',
 u'data/skins/old_caucasian_male/old_caucasian_male.mhmat',
 u'data/skins/toon01/toon01.mhmat',
 u'data/skins/young_african_female/young_african_female.mhmat',
 u'data/skins/young_african_male/young_african_male.mhmat',
 u'data/skins/young_asian_female/young_asi

In [24]:
skins = _listDataFiles('skins','mhmat')
import material
for skin in skins:
    print(skin)
    mat = material.fromFile(skin)
    mhmat_file = Path(skin.replace('data/skins/',''))
    outsubdir = outdir.joinpath('skins',mhmat_file.dirname())
    outsubdir.makedirs_p()
    outfile = outsubdir.joinpath(mhmat_file.basename()).replace('.mhmat','.json')

    mtllib = material_to_mtl(mat, texdir=os.path.dirname(outfile))
    mtl = create_materials({mat.name:0}, mtllib, '')
    data = mtl[mtl.keys()[0]]
    json.dump(data,open(outfile,'w'))
    
    # and the thumbnail
    thumbnail = Path(mat.filename.replace('.mhmat','.thumb'))
    if thumbnail.isfile():
        copyAndCompress(thumbnail,outfile.replace('.json','.thumb.png'))

data/skins/default.mhmat
data/skins/middleage_african_female/middleage_african_female.mhmat
data/skins/middleage_african_male/middleage_african_male.mhmat
data/skins/middleage_asian_female/middleage_asian_female.mhmat
data/skins/middleage_asian_male/middleage_asian_male.mhmat
data/skins/middleage_caucasian_female/middleage_caucasian_female.mhmat
data/skins/middleage_caucasian_male/middleage_caucasian_male.mhmat
data/skins/old_african_female/old_african_female.mhmat
data/skins/old_african_male/old_african_male.mhmat
data/skins/old_asian_female/old_asian_female.mhmat
data/skins/old_asian_male/old_asian_male.mhmat
data/skins/old_caucasian_female/old_caucasian_female.mhmat
data/skins/old_caucasian_male/old_caucasian_male.mhmat
data/skins/toon01/toon01.mhmat
data/skins/young_african_female/young_african_female.mhmat
data/skins/young_african_male/young_african_male.mhmat
data/skins/young_asian_female/young_asian_female.mhmat
data/skins/young_asian_male/young_asian_male.mhmat
data/skins/young

In [25]:
# test makehuman2mtl material_to_mtl=>create_materials
mat = material.fromFile(skin)
mtllib = material_to_mtl(mat, texdir=os.path.dirname(outfile))
mtl = create_materials({mat.name:0}, mtllib, '')
print(mtl)
keys = [
   
    'colorDiffuse',
    'colorSpecular',
    'colorEmissive',
    'opacity',
    'wireframe',
    'specularCoef',
    
    'mapDiffuse',
    
    'shading',
    'transparent'
]
for k in keys:
    assert k in mtl[mat.name], k

{u'young_caucasian_male_detailed': {'opacity': 1.0, 'specularCoef': 0.5, 'colorEmissive': [0.1961, 0.102, 0.102], 'wireframe': 0, 'colorDiffuse': [1.0, 1.0, 1.0], 'mapDiffuse': 'textures/young_caucasian_male_special_suit.png', 'colorSpecular': [0.02745, 0.02745, 0.02745], 'shading': 'phong', 'transparent': True}}
